In [1]:
import numpy as np
import pandas as pd
import os
from sklearn.model_selection import KFold, StratifiedKFold
import tensorflow as tf
#from tensorflow.keras.preprocessing.image import ImageDataGenerator
import keras
from keras_preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Activation, Dropout, Flatten, Dense, Conv2D, MaxPooling2D
tf.keras.utils.to_categorical

<function tensorflow.python.keras.utils.np_utils.to_categorical(y, num_classes=None, dtype='float32')>

In [2]:
train_data = pd.read_csv('classification.csv', dtype=str)
#data = pd.read_csv('./Hand_Annotations_2.csv',dtype=str,names=colnames
Y = train_data[['label']]

kf = KFold(n_splits = 10)
                         
skf = StratifiedKFold(n_splits = 10, random_state = 7, shuffle = True)
image_shape = (150,150,3)

In [3]:
idg = ImageDataGenerator(#width_shift_range=0.1,
                         #height_shift_range=0.1,
                         #zoom_range=0.3,
                         fill_mode='nearest',
                         horizontal_flip = True)
                         #rescale=1./255)

In [4]:
def get_model_name(k):
    return 'model_'+str(k)+'.h5'

In [5]:
#model = tf.keras.models.Sequential()
model = Sequential()

model.add(Conv2D(filters=32, kernel_size=(3,3),input_shape=(150,150,3), activation='relu',))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(filters=64, kernel_size=(3,3),input_shape=(150,150,3), activation='relu',))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(filters=64, kernel_size=(3,3),input_shape=(150,150,3), activation='relu',))
model.add(MaxPooling2D(pool_size=(2, 2)))


model.add(Flatten())


model.add(Dense(128))
model.add(Activation('relu'))

# Dropouts help reduce overfitting by randomly turning neurons off during training.
# Here we say randomly turn off 50% of neurons.
model.add(Dropout(0.5))

# Last layer, not binary!
model.add(Dense(3))
model.add(Activation('sigmoid'))

model.compile(loss='categorical_crossentropy', #not binary for the poses
              optimizer='rmsprop',
              metrics=['accuracy'])

In [8]:
import warnings
warnings.filterwarnings('ignore')

In [13]:
VALIDATION_ACCURACY = []
VALIDATION_LOSS = []

save_dir = 'kfoldModels/'
fold_var = 1

for train_index, val_index in kf.split(np.zeros(707),Y):
    training_data = train_data.iloc[train_index]
    validation_data = train_data.iloc[val_index]

    train_data_generator = idg.flow_from_dataframe(training_data, target_size=image_shape[:2],
                               batch_size = 16,
                               x_col = "filename", y_col = "label",
                               class_mode = "categorical", shuffle = True)
    valid_data_generator  = idg.flow_from_dataframe(validation_data, target_size=image_shape[:2],
                            batch_size = 16,
                            x_col = "filename", y_col = "label",
                            class_mode = "categorical", shuffle = True)
    
  

    # CREATE NEW MODEL
    #model = create_new_model()

    # COMPILE NEW MODEL
    model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

    # CREATE CALLBACKS
    checkpoint = tf.keras.callbacks.ModelCheckpoint(save_dir+get_model_name(fold_var), 
                            monitor='val_accuracy', verbose=1, 
                            save_best_only=True, mode='max')
    callbacks_list = [checkpoint]
    # There can be other callbacks, but just showing one because it involves the model name
    # This saves the best model
    # FIT THE MODEL
    history = model.fit(train_data_generator,
                epochs=10,
                #steps_per_epoch=150,
                callbacks=callbacks_list,
                validation_data=valid_data_generator)
    #PLOT HISTORY

    # LOAD BEST MODEL to evaluate the performance of the model
    model.load_weights("kfoldModels/model_"+str(fold_var)+".h5")

    results = model.evaluate(valid_data_generator)
    results = dict(zip(model.metrics_names,results))

    VALIDATION_ACCURACY.append(results['accuracy'])
    VALIDATION_LOSS.append(results['loss'])

    tf.keras.backend.clear_session()

    fold_var += 1



Found 636 validated image filenames belonging to 3 classes.
Found 71 validated image filenames belonging to 3 classes.
Epoch 1/10
40/40 [==============================] - 8s 184ms/step - loss: 0.2077 - accuracy: 0.9607 - val_loss: 0.0193 - val_accuracy: 0.9859

Epoch 00001: val_accuracy improved from -inf to 0.98592, saving model to kfoldModels/model_1.h5
Epoch 2/10
40/40 [==============================] - 7s 179ms/step - loss: 0.0935 - accuracy: 0.9764 - val_loss: 0.0024 - val_accuracy: 1.0000

Epoch 00002: val_accuracy improved from 0.98592 to 1.00000, saving model to kfoldModels/model_1.h5
Epoch 3/10
40/40 [==============================] - 7s 178ms/step - loss: 0.0343 - accuracy: 0.9874 - val_loss: 5.4229e-04 - val_accuracy: 1.0000

Epoch 00003: val_accuracy did not improve from 1.00000
Epoch 4/10
40/40 [==============================] - 7s 178ms/step - loss: 0.0351 - accuracy: 0.9843 - val_loss: 0.0018 - val_accuracy: 1.0000

Epoch 00004: val_accuracy did not improve from 1.00000


40/40 [==============================] - 7s 181ms/step - loss: 0.0170 - accuracy: 0.9969 - val_loss: 0.0171 - val_accuracy: 0.9859

Epoch 00007: val_accuracy did not improve from 1.00000
Epoch 8/10
40/40 [==============================] - 7s 181ms/step - loss: 0.0316 - accuracy: 0.9921 - val_loss: 4.9039e-05 - val_accuracy: 1.0000

Epoch 00008: val_accuracy did not improve from 1.00000
Epoch 9/10
40/40 [==============================] - 7s 180ms/step - loss: 0.0170 - accuracy: 0.9937 - val_loss: 0.0712 - val_accuracy: 0.9859

Epoch 00009: val_accuracy did not improve from 1.00000
Epoch 10/10
40/40 [==============================] - 7s 180ms/step - loss: 0.0149 - accuracy: 0.9937 - val_loss: 0.1048 - val_accuracy: 0.9859

Epoch 00010: val_accuracy did not improve from 1.00000
5/5 [==============================] - 0s 35ms/step - loss: 3.8834e-05 - accuracy: 1.0000
Found 636 validated image filenames belonging to 3 classes.
Found 71 validated image filenames belonging to 3 classes.
Epoch

40/40 [==============================] - 7s 182ms/step - loss: 0.0501 - accuracy: 0.9906 - val_loss: 1.2970e-04 - val_accuracy: 1.0000

Epoch 00003: val_accuracy did not improve from 1.00000
Epoch 4/10
40/40 [==============================] - 7s 183ms/step - loss: 0.0244 - accuracy: 0.9953 - val_loss: 8.3396e-06 - val_accuracy: 1.0000

Epoch 00004: val_accuracy did not improve from 1.00000
Epoch 5/10
40/40 [==============================] - 7s 182ms/step - loss: 0.0285 - accuracy: 0.9953 - val_loss: 4.9729e-04 - val_accuracy: 1.0000

Epoch 00005: val_accuracy did not improve from 1.00000
Epoch 6/10
40/40 [==============================] - 7s 182ms/step - loss: 0.0305 - accuracy: 0.9953 - val_loss: 0.0041 - val_accuracy: 1.0000

Epoch 00006: val_accuracy did not improve from 1.00000
Epoch 7/10
40/40 [==============================] - 7s 182ms/step - loss: 0.0123 - accuracy: 0.9953 - val_loss: 2.5374e-07 - val_accuracy: 1.0000

Epoch 00007: val_accuracy did not improve from 1.00000
Epoch

In [12]:
VALIDATION_ACCURACY

[0.7746478915214539,
 0.9295774698257446,
 0.7605633735656738,
 0.9014084339141846,
 0.98591548204422,
 0.9718309640884399,
 0.98591548204422,
 0.9714285731315613,
 0.9714285731315613,
 0.9857142567634583]